In [1]:
# Cài detectron2 từ source GitHub (tương thích mọi môi trường Kaggle)
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\lequo\appdata\local\temp\pip-req-build-p1engh6q
  Resolved https://github.com/facebookresearch/detectron2.git to commit 9604f5995cc628619f0e4fd913453b4d7d61db3f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for detectron2: filename=detectron2-0.6-cp39-cp39-win_amd64.whl size=890259 sha256=a9cd12bbe749c16ab7415bbf9a710e0466c6c6899496d5c81f3b6b34cc100241
  Stored in directory: C:\Users\lequo\AppData\Local\Temp\pip-ephem-wheel-cache-300s00hr\wheels\59\b4\83\84bfca751fa4dcc59998468be8688eb50e97408a83af171d42
Successfully built detectron2


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\lequo\AppData\Local\Temp\pip-req-build-p1engh6q'


In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:
import os
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, HookBase
from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader, DatasetCatalog
import torch
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.events import get_event_storage
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Đường dẫn đến dataset của bạn
dataset_dir = "/kaggle/input/data-for-skin-problem"  # Thay đổi đường dẫn thực tế nếu cần

In [ ]:
# Đăng ký tập TRAIN
train_ann_file = os.path.join(dataset_dir, "train", "_annotations.coco.json")
train_images_dir = os.path.join(dataset_dir, "train")
register_coco_instances("skin_train_custom", {}, train_ann_file, train_images_dir)

# Đăng ký tập VALID (thư mục là "valid")
val_ann_file = os.path.join(dataset_dir, "valid", "_annotations.coco.json")
val_images_dir = os.path.join(dataset_dir, "valid")
register_coco_instances("skin_valid_custom", {}, val_ann_file, val_images_dir)

# Đăng ký tập TEST
test_ann_file = os.path.join(dataset_dir, "test", "_annotations.coco.json")
test_images_dir = os.path.join(dataset_dir, "test")
register_coco_instances("skin_test_custom", {}, test_ann_file, test_images_dir)

In [ ]:
dataset_len = len(DatasetCatalog.get("skin_train_custom"))
print("Số ảnh trong skin_train_custom:", dataset_len)

# **⚙️ Config**

In [28]:

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("skin_train_custom",)
cfg.DATASETS.TEST = ("skin_valid_custom",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 8  # Có thể tăng nếu GPU dư RAM
cfg.SOLVER.BASE_LR = 0.000375  # Tăng theo batch size
cfg.SOLVER.MAX_ITER = 67000  # ~30 epoch nếu ~500 iter/epoch
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.STEPS = (10000, 13000)
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.AMP.ENABLED = True  # Mixed Precision = nhanh hơn
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13  # Điều chỉnh nếu loại class rỗng
cfg.MODEL.DEVICE = "cuda"
cfg.TEST.EVAL_PERIOD = 0  # Eval bằng hook
cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
# 🔨 Build model & reset head
model = build_model(cfg)
checkpointer = DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR)
checkpointer.load(cfg.MODEL.WEIGHTS)
model.roi_heads.box_predictor.cls_score.reset_parameters()
model.roi_heads.box_predictor.bbox_pred.reset_parameters()
model.to(torch.device(cfg.MODEL.DEVICE))

# 📊 Custom hook: log loss + eval mỗi epoch
class EpochEvalHook(HookBase):
    def __init__(self, dataset_len, batch_size, cfg):
        self.iter_per_epoch = dataset_len / batch_size
        self.cfg = cfg
        self.last_epoch = -1

    def after_step(self):
        storage = get_event_storage()
        current_iter = storage.iter
        current_epoch = int(current_iter / self.iter_per_epoch)

        # Chỉ chạy khi qua epoch mới
        if current_epoch > self.last_epoch:
            self.last_epoch = current_epoch
            try:
                loss = storage.history("total_loss").latest()
                print(f"Epoch: {current_epoch} - Iteration: {current_iter} - Loss: {loss:.4f}")
            except:
                print(f"Epoch: {current_epoch} - Iteration: {current_iter} - Loss not available")

            evaluator = COCOEvaluator("skin_valid_custom", output_dir=self.cfg.OUTPUT_DIR)
            val_loader = build_detection_test_loader(self.cfg, "skin_valid_custom")
            results = inference_on_dataset(self.trainer.model, val_loader, evaluator)
            print(f"Eval: {results}")

# 🧠 Custom trainer dùng lại mô hình build sẵn
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_model(cls, cfg):
        return model

trainer = MyTrainer(cfg)
trainer.register_hooks([EpochEvalHook(dataset_len, cfg.SOLVER.IMS_PER_BATCH, cfg)])
trainer.resume_or_load(resume=False)
trainer.train()

Epoch: 0 - Iteration: 0 - Loss: 3.2697


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Loading and preparing results...
DONE (t=0.07s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.

In [1]:
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer

# Cấu hình mô hình
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13
cfg.MODEL.WEIGHTS = "/kaggle/input/test_model/pytorch/default/1/model_0044999.pth"  # Đường dẫn tới file .pth của bạn
cfg.MODEL.DEVICE = "cpu"  # Hoặc "cuda" nếu bạn muốn dùng GPU
predictor = DefaultPredictor(cfg)

# Đọc ảnh mẫu (OpenCV trả về ảnh ở định dạng BGR)
im = cv2.imread("/kaggle/input/tesst-image/test_image.jpg")  # Thay đổi đường dẫn cho phù hợp

# Chuyển đổi ảnh từ BGR sang RGB
im_rgb = im[:, :, ::-1]

# Dự đoán
outputs = predictor(im)

# Tạo Visualizer với ảnh đã chuyển sang RGB
v = Visualizer(im_rgb, scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Hiển thị ảnh kết quả (vẫn ở định dạng RGB)
plt.figure(figsize=(12, 12))
plt.imshow(out.get_image())
plt.axis("off")
plt.show()

ModuleNotFoundError: No module named 'detectron2'